<a href="https://colab.research.google.com/github/KoshKath/DataScience_Lab2/blob/main/Lab2_ml_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ml_module.py
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# загрузка файла
def load_data(file_path):
  try:
    return pd.read_csv(file_path)
  except FileNotFoundError:
    print(f"Файл {file_path} не найден.")
    return None
  except Exception:
    print(f"Ошибка при чтении файла {file_path}.")
    return None

# исследование данных датасета
def data_quality(df):
  df.info()
  print("---------------------------------------------------------")
  print("Датасет 'Приюты':")
  print(df.info)
  print("---------------------------------------------------------")
  print("Краткая аналитика сведений датасета:")
  print(df.describe())
  print("---------------------------------------------------------")

  # распределение данных датасета в зависимости от сезона
  sns.pairplot(df, hue='season')
  plt.show()
  # распределение данных датасета в зависимости от штата
  sns.pairplot(df, hue='state')
  plt.show()
  #гистограмма среднего возраста
  plt.hist(df['average_age'], bins=40)
  plt.ylabel('Количество приютов')
  plt.xlabel('Средний возраст')
  plt.show()
  #гистограмма количесвто приютов по штатам
  plt.hist(df['state'], bins=40)
  plt.ylabel('Количество приютов')
  plt.xlabel('Штат')
  plt.show()
  #диаграмма рассеивания
  x_values = df['average_age']
  y_values = df['female_percentage']
  plt.scatter(x_values, y_values)
  plt.xlabel('Средний возраст находящихся в приюте')
  plt.ylabel('% женщин в приюте')
  plt.show()
  return

# предобработка
def preprocess_data(df, target_column):
  # выполнение метода, если датасет найден
  if df is not None:
    # в X не должен попасть y
    X = df.drop(columns=[target_column], axis=1)
    y = df[target_column]
  else:
    print("Датасет не найден. Проверьте загрузку файла.")
  # обработка колонки Дата: создание года и месяца, затем удаление первоначальной даты
  if X.columns.isin(['date']).any():
     X['date'] = pd.to_datetime(X['date'], errors='coerce')
     X['date_year'] = X['date'].dt.year
     X['date_month'] = X['date'].dt.month
     X = X.drop(columns='date')
  # определение категориальных признаков
  numeric_features = ['id', 'total_capacity', 'occupied_beds', 'available_beds', 'average_age', 'male_percentage', 'female_percentage']
  categorical_features = ['shelter_name', 'city', 'state', 'season', 'notes']
  # создание препроцессора
  numeric_transformer = StandardScaler()
  categorical_transformer = OneHotEncoder(drop='first')

  preprocessor = ColumnTransformer(
      transformers=[
          ('num', numeric_transformer, numeric_features),
          ('cat', categorical_transformer, categorical_features)
      ])
  # применение препроцессора к данным
  X_processed = preprocessor.fit_transform(X)
  return X_processed, y, preprocessor

# обучение модели
def train_model(X, y):
  try:
    model = LinearRegression()
    model.fit(X, y)
    return model
  except Exception as e:
    print(f"Ошибка при обучении модели: {e}")

# предсказание на тестовых данных
def predict(model, X):
  if model is None:
    print("Ошибка: модель отсутствует.")
    return

  try:
    return model.predict(X)
  except Exception as e:
    print(f"Ошибка в предсказании модели: {e}")

# оценка модели
def evaluate_model(y_true, y_pred):
  mse = mean_squared_error(y_true, y_pred)
  r2 = r2_score(y_true, y_pred)
  return mse, r2

# визуализация истинных и предсказанных значений
def plot_predictions(y_true, y_pred, num_points=70):
  plt.figure(figsize=(10, 6))
  plt.scatter(range(num_points), y_true[:num_points], color='blue', label='Истинные значения')
  plt.scatter(range(num_points), y_pred[:num_points], color = 'red', label='Предсказанные значения')
  plt.xlabel('Индекс')
  plt.ylabel('Значение occupancy_rate, %')
  plt.title(f'Истинные и предсказанные значения occupancy_rate (%) (первые {num_points} точек)')
  plt.legend()
  plt.show()
  return


